# Uvod v načrtovanje pozivov
Načrtovanje pozivov je postopek oblikovanja in optimizacije pozivov za naloge obdelave naravnega jezika. Vključuje izbiro ustreznih pozivov, prilagajanje njihovih parametrov ter ocenjevanje njihove učinkovitosti. Načrtovanje pozivov je ključno za doseganje visoke natančnosti in učinkovitosti pri NLP modelih. V tem poglavju bomo raziskali osnove načrtovanja pozivov s pomočjo OpenAI modelov za raziskovanje.


### Vaja 1: Tokenizacija
Raziščite tokenizacijo s tiktoken, odprtokodnim hitrim tokenizatorjem podjetja OpenAI
Za več primerov si oglejte [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst).


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Vaja 2: Preverite nastavitev ključa za Github Models

Zaženite spodnjo kodo, da preverite, ali je vaš Github Models endpoint pravilno nastavljen. Koda preizkusi preprost osnovni poziv in preveri dokončanje. Vnos `oh say can you see` naj bi se dopolnil v stilu `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Vaja 3: Izmišljotine
Raziščite, kaj se zgodi, ko LLM-u naročite, naj vrne odgovore na vprašanje o temi, ki morda ne obstaja, ali o temah, o katerih morda nima podatkov, ker so zunaj njegovega predhodno naučenega nabora podatkov (novejše teme). Oglejte si, kako se odziv spremeni, če poskusite z drugačnim pozivom ali z drugim modelom.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Vaja 4: Na podlagi navodil
Uporabi spremenljivko "text" za nastavitev glavne vsebine
in spremenljivko "prompt" za podajanje navodila, povezanega s to glavno vsebino.

Tukaj modelu naročimo, naj povzame besedilo za učenca drugega razreda.


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Vaja 5: Kompleksno navodilo
Poskusi zahtevo, ki vključuje sistemska, uporabniška in asistentova sporočila
Sistem določi kontekst asistenta
Uporabniška in asistentova sporočila ustvarjajo kontekst večkratnega pogovora

Opazuj, kako je osebnost asistenta v sistemskem kontekstu nastavljena na "sarkastično".
Poskusi uporabiti drugačen kontekst osebnosti. Ali pa preizkusi drugačno zaporedje vhodnih/izhodnih sporočil


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Vaja: Raziskujte svojo intuicijo
Zgornji primeri vam ponujajo vzorce, ki jih lahko uporabite za ustvarjanje novih pozivov (preprostih, zahtevnih, navodilnih itd.) – poskusite ustvariti še druge vaje, s katerimi boste raziskali nekatere druge ideje, o katerih smo govorili, kot so primeri, namigi in še več.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da se zavedate, da lahko samodejni prevodi vsebujejo napake ali netočnosti. Izvirni dokument v svojem izvirnem jeziku naj velja za avtoritativni vir. Za ključne informacije priporočamo strokovni človeški prevod. Ne prevzemamo odgovornosti za morebitna nesporazume ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
